In [1]:
import pandas as pd
import requests
import os

### Acquire exercises

### 1. Create a dataframe named 'items' that has all of the data for items

In [2]:
base_url = 'https://python.zach.lol'
response = requests.get(base_url)
response.json()

{'api': '/api/v1', 'help': '/documentation'}

In [3]:
doc_url = base_url + '/documentation'

In [4]:
requests.get(doc_url).json().keys()

dict_keys(['payload', 'status'])

In [5]:
print(requests.get(doc_url).json()['payload'])


The API accepts GET requests for all endpoints, where endpoints are prefixed
with

    /api/{version}

Where version is "v1"

Valid endpoints:

- /stores[/{store_id}]
- /items[/{item_id}]
- /sales[/{sale_id}]

All endpoints accept a `page` parameter that can be used to navigate through
the results.



In [6]:
api_url = base_url + '/api/v1/'

In [7]:
response = requests.get(api_url + 'items')
response.ok

True

In [8]:
data = response.json()

print(type(data))

<class 'dict'>


In [9]:
data['payload'].keys()

dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [10]:
items = data['payload']['items']
print(len(items))
type(items)

20


list

In [11]:
data['payload']['next_page']

'/api/v1/items?page=2'

In [12]:
# getting page 2

response = requests.get(base_url + data['payload']['next_page'])

In [13]:
data = response.json()

In [14]:
items.extend(data['payload']['items'])

In [15]:
data['payload']['next_page']

'/api/v1/items?page=3'

In [16]:
response = requests.get(base_url + data['payload']['next_page'])

data = response.json()

In [17]:
items.extend(data['payload']['items'])
len(items)

50

In [18]:
data['payload']['next_page'] == None

True

In [19]:
# Use the list of dictionaries, to create a DataFrame of items

items_df = pd.DataFrame(items)
print(f'The items_df has the shape {items_df.shape}.\n')
items_df.head(2)

The items_df has the shape (50, 6).



,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925


### 2. Do the same thing for stores 

In [20]:
api_url = base_url + '/api/v1/'
response = requests.get(api_url + 'stores')
data = response.json()

In [21]:
data['payload']['max_page']

1

In [22]:
data['payload'].keys()

dict_keys(['max_page', 'next_page', 'page', 'previous_page', 'stores'])

In [23]:
stores = data['payload']['stores'][:2]
stores_df = pd.DataFrame(stores)

In [24]:
print(f"My stores_df has the shape {stores_df.shape}")
stores_df.head()

My stores_df has the shape (2, 5)


,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251


### 3. Extract the data for sales

In [25]:
api_url = base_url + '/api/v1/'
response = requests.get(api_url + 'sales')
data = response.json()
data['payload']['max_page']

183

### 4. Save the data in your files to local csv files

In [26]:
# Build a helper function

def get_df(name):
    """
    This function takes in the string
    'items', 'stores', or 'sales' and
    returns a df containing all pages and
    creates a .csv file for future use.
    """
    base_url = 'https://python.zach.lol'
    api_url = base_url + '/api/v1/'
    response = requests.get(api_url + name)
    data = response.json()
    
    # create list from 1st page
    my_list = data['payload'][name]
    
    # loop through the pages and add to list
    while data['payload']['next_page'] != None:
        response = requests.get(base_url + data['payload']['next_page'])
        data = response.json()
        my_list.extend(data['payload'][name])
    
    # Create DataFrame from list
    df = pd.DataFrame(my_list)
    
    # Write DataFrame to csv file for future use
    df.to_csv(name + '.csv')
    return df

In [27]:
items_df = get_df('items')
print(items_df.shape)
items_df.head()

(50, 6)


,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [28]:
stores_df = get_df('stores')
print(stores_df.shape)
stores_df.head()

(10, 5)


,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


In [29]:
sales_df = get_df('sales')
print(sales_df.shape)
sales_df.head()

(913000, 5)


,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


### 5. Merge dataframes

In [30]:
df = pd.merge(sales_df, stores_df, left_on='store', right_on='store_id').drop(columns={'store'})
df.head(2)

,item,sale_amount,sale_date,sale_id,store_address,store_city,store_id,store_state,store_zipcode
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253


In [31]:
df = pd.merge(df, items_df, left_on='item', right_on='item_id').drop(columns={'item'})
df.head(2)

,sale_amount,sale_date,sale_id,store_address,store_city,store_id,store_state,store_zipcode,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013


In [32]:
df.shape

(913000, 14)

### 6. Acquire the Open Power Systems Data for Germany

In [33]:
url = 'https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv'
df = pd.read_csv(url)
df.head()

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.184,NaN,NaN,NaN
1,2006-01-02,1380.521,NaN,NaN,NaN
2,2006-01-03,1442.533,NaN,NaN,NaN
3,2006-01-04,1457.217,NaN,NaN,NaN
4,2006-01-05,1477.131,NaN,NaN,NaN


In [34]:
def opsd_germany_daily():
    """
    This function uses or creates the 
    opsd_germany_daily csv and returns a df.
    """
    if os.path.isfile('opsd_germany_daily.csv'):
        df = pd.read_csv('opsd_germany_daily.csv', index_col=0)
    else:
        url = 'https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv'
        df = pd.read_csv(url)
        df.to_csv('opsd_germany_daily.csv')
    return df

In [35]:
gdf = opsd_germany_daily()
gdf.head(2)

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.184,NaN,NaN,NaN
1,2006-01-02,1380.521,NaN,NaN,NaN
